In [ ]:
import os
import time
import logging
import kfp
from google.cloud import bigquery, storage
from google.cloud import aiplatform as vertex_ai
from google_cloud_pipeline_components.experimental.custom_job import utils
from kfp.v2 import compiler, dsl
from kfp.v2.dsl import component
from typing import NamedTuple
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output, Metrics,
                        OutputPath, component)

from google_cloud_pipeline_components.experimental.custom_job import utils

In [ ]:
logging.basicConfig(level=logging.INFO)

## Load Params and Resource Config

In [ ]:
from config.gcp_resource import *

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    
if SERVICE_ACCOUNT == "" or SERVICE_ACCOUNT is None or SERVICE_ACCOUNT == "[your-service-account]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.account)' 2>/dev/null
    SERVICE_ACCOUNT = shell_output[0]
    
if GCS_BUCKET == "" or GCS_BUCKET is None or GCS_BUCKET == "[your-bucket-name]":
    # Get your bucket name to GCP projet id
    GCS_BUCKET = PROJECT_ID
    # Try to create the bucket if it doesn'exists
    ! gsutil mb -l $REGION gs://$BUCKET
    print("")
    
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [ ]:
print("Train machine type", TRAIN_COMPUTE)
print("Deploy machine type", DEPLOY_COMPUTE)
print("Deployment:", DEPLOY_IMAGE)
print('PIPELINE_ROOT: {}'.format(PIPELINE_ROOT))
print('MODULE_ROOT: {}'.format(MODULE_ROOT))
print('DATA_ROOT: {}'.format(DATA_ROOT))
print('SERVING_MODEL_DIR: {}'.format(SERVING_MODEL_DIR))

Train machine type n1-standard-4
Deploy machine type n1-standard-4
Deployment: us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest
PIPELINE_ROOT: gs://mle_airbus_dataset/airbusmlepipeline/pipeline_root
MODULE_ROOT: gs://mle_airbus_dataset/airbusmlepipeline/pipeline_module
DATA_ROOT: gs://mle_airbus_dataset/airbusmlepipeline/data
SERVING_MODEL_DIR: gs://mle_airbus_dataset/airbusmlepipeline/serving_model


## Model Evaluation Component

In [18]:
%%writefile ./src/evaluation/eval_component.py

import tensorflow.keras.backend as K
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy,BinaryCrossentropy 
from google.cloud import storage
from src.models.preprocessing import Augment
from src.utils.common import *
from src.utils.dataset import *

def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

def dice_p_bce(in_gt, in_pred):
    return 1e-3*binary_crossentropy(in_gt, in_pred) - dice_coef(in_gt, in_pred)

def true_positive_rate(y_true, y_pred):
    return K.sum(K.flatten(y_true)*K.flatten(K.round(y_pred)))/K.sum(y_true)

logging.basicConfig(level=logging.INFO)

parser = argparse.ArgumentParser()
parser.add_argument('--test_filepath', dest='test_filepath',
                    default='', type=str,
                    help='Validation data file path')
parser.add_argument('--model_filepath', dest='model_filepath',
                    default='', type=str,
                    help='Model file path')
parser.add_argument('--output', dest='metrics',
                    default='', type=str,
                    help='Metrics output')
args = parser.parse_args()

test_filepath: str,
model_filepath: str,
metrics: Output[Metrics]

IMG_SHAPE=(128,128)
GCS_BUCKET="mle_airbus_dataset"
BATCH_SIZE = 16
EDGE_CROP = 16
NB_EPOCHS = 10
GAUSSIAN_NOISE = 0.1
UPSAMPLE_MODE = 'SIMPLE'
# downsampling inside the network
NET_SCALING = None
# downsampling in preprocessing
IMG_SCALING = (1, 1)
# number of validation images to use
VALID_IMG_COUNT = 10
# maximum number of steps_per_epoch in training
MAX_TRAIN_STEPS = 200
AUGMENT_BRIGHTNESS = False
N_SAMPLE = 100
bucket = storage.Client().bucket(GCS_BUCKET)

blob = bucket.blob("test.parquet")
blob.download_to_filename("test.parquet")

valid_df = pd.read_parquet(f"test.parquet")
validation = tf.data.Dataset.from_tensor_slices((valid_df['ImageId'].values, valid_df['EncodedPixels'].values))
validation = validation.shuffle(buffer_size=10)
validation = validation.map(lambda x, y: parse_db_to_img("gs://mle_airbus_dataset/train_v2/" + x, y))
validation = validation.batch(BATCH_SIZE)
validation = validation.map(Augment(resize_shape=IMG_SHAPE, train=False))
validation = validation.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
model_eval = tf.keras.models.load_model(args.model_filepath, compile=False)
model_eval.compile(optimizer=Adam(1e-4, decay=1e-6), loss=dice_p_bce, metrics=[dice_coef, 'binary_accuracy', true_positive_rate])
result = model_eval.evaluate(validation)
metrics.log_metric("dice_coef", (result[1]))
metrics.log_metric("binary_accuracy", (result[2]))
metrics.log_metric("true_positive_rate", (result[3]))
    import numpy as np
    import google.cloud.aiplatform as aip

Overwriting ./src/evaluation/eval_component.py


## Model Evaluation Unit Test

In [8]:
@component(
    base_image="python:3.7",
    output_component_file="./build/model_eval_test_component.yaml")
def model_eval_test_component(
    metrics: Input[Metrics]
):
    """
    Unit test component that checks if the output metrics passed
    thresholds
    """
    import logging
    
    metrics_thresholds = {
        'dice_coef': 0.1,
        'binary_accuracy': 0.8,
        'true_positive_rate': 0.3
    }
    
    for k, v in metrics.metadata.items():
        assert v >= metrics_thresholds[k]
        logging.info(f"{k}:{v}, threshold: {metrics_thresholds[k]}. Passed.")
